In [1]:
import torch
import pandas as pd
import torch.nn as nn
import numpy as np
model = torch.hub.load('pytorch/vision:v0.6.0', 'vgg11', pretrained=True)
model.eval()
new_classifier = torch.nn.Sequential(*list(model.classifier.children())[:-1])
model.classifier = new_classifier

Using cache found in C:\Users\cpsch/.cache\torch\hub\pytorch_vision_v0.6.0


In [2]:
data = pd.read_csv("picture_urls.csv")
data.shape

(92893, 1)

In [3]:
from PIL import Image
from torchvision import transforms

def image_to_vector(url):
    
    filename = "images/" + url.split("/")[-1]
    try:
        input_image = Image.open(filename)
        preprocess_images = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
        input_tensor = preprocess_images(input_image)
        input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

        # move the input and model to GPU for speed if available
        if torch.cuda.is_available():
            input_batch = input_batch.to('cuda')
            model.to('cuda')

        with torch.no_grad():
            output = model(input_batch)
        return output
    except:
        return torch.zeros(4096)


In [5]:
data["image_tensor"] = data["picture_url"].apply(lambda k: image_to_vector(k))

In [6]:
data.image_tensor

0        [[tensor(0.), tensor(0.), tensor(0.), tensor(0...
1        [[tensor(0.), tensor(0.), tensor(0.), tensor(0...
2        [[tensor(0.), tensor(0.), tensor(0.), tensor(0...
3        [[tensor(0.), tensor(0.), tensor(0.), tensor(0...
4        [[tensor(1.7615), tensor(0.), tensor(0.), tens...
                               ...                        
92888    [[tensor(0.), tensor(0.), tensor(0.), tensor(0...
92889    [[tensor(0.), tensor(0.), tensor(0.), tensor(0...
92890    [[tensor(0.), tensor(0.), tensor(0.), tensor(0...
92891    [[tensor(0.), tensor(0.), tensor(0.), tensor(0...
92892    [[tensor(0.), tensor(0.8798), tensor(0.), tens...
Name: image_tensor, Length: 92893, dtype: object

In [7]:
image_data = torch.vstack(tuple(data.image_tensor))

In [14]:
torch.save(image_data, "image_tensor.pt")